# Implementation of Quantum Restricted Boltzmann Machine Algorithm Neural Network

## Import Necessary Libraries

In [26]:
from collections import Counter
from functools import reduce
from typing import Any, Dict, List, Tuple

import numpy as np
from scipy.optimize import minimize, fmin_bfgs
from copy import deepcopy
import numpy as np
from pyquil import Program
from pyquil.api import QuantumComputer, WavefunctionSimulator
from pyquil.gates import H, MEASURE
from pyquil.paulis import exponential_map, PauliSum
from scipy import optimize, ufunc
import funcsigs 
import pyquil.quil as pq
from pyquil import get_qc, Program
import pyquil.api as api
from pyquil.paulis import *
from pyquil.gates import *

import warnings
warnings.filterwarnings('ignore')

## Helper Classes

### Set Up the Variational Quantum Eigensolver from grove to perform VQE for the Variational QRBM

In [27]:
class OptResults(dict):
    """
    Object for holding optimization results from VQE.
    """
    def __getattr__(self, name):
        try:
            return self[name]
        except KeyError:
            raise AttributeError(name)

    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


class VQE(object):
    """
    The Variational-Quantum-Eigensolver algorithm
    VQE is an object that encapsulates the VQE algorithm (functional
    minimization). The main components of the VQE algorithm are a minimizer
    function for performing the functional minimization, a function that takes a
    vector of parameters and returns a pyQuil program, and a
    Hamiltonian of which to calculate the expectation value.
    Using this object:
        1) initialize with `inst = VQE(minimizer)` where `minimizer` is a
        function that performs a gradient free minization--i.e
        scipy.optimize.minimize(. , ., method='Nelder-Mead')
        2) call `inst.vqe_run(variational_state_evolve, hamiltonian,
        initial_parameters)`. Returns the optimal parameters and minimum
        expecation
    :param minimizer: function that minimizes objective f(obj, param). For
                      example the function scipy.optimize.minimize() needs
                      at least two parameters, the objective and an initial
                      point for the optimization.  The args for minimizer
                      are the cost function (provided by this class),
                      initial parameters (passed to vqe_run() method, and
                      jacobian (defaulted to None).  kwargs can be passed
                      in below.
    :param minimizer_args: (list) arguments for minimizer function. Default=None
    :param minimizer_kwargs: (dict) arguments for keyword args.
                              Default=None
    """

    def __init__(self, minimizer, minimizer_args=[], minimizer_kwargs={}):
        self.minimizer = minimizer
        self.minimizer_args = minimizer_args
        self.minimizer_kwargs = minimizer_kwargs
        self.n_qubits = None

    def vqe_run(self, variational_state_evolve, hamiltonian, initial_params,
                gate_noise=None, measurement_noise=None,
                jacobian=None, qc=None, disp=None, samples=None, return_all=False):
        """
        functional minimization loop.
        :param variational_state_evolve: function that takes a set of parameters
                                        and returns a pyQuil program.
        :param hamiltonian: (PauliSum) object representing the hamiltonian of
                            which to take the expectation value.
        :param initial_params: (ndarray) vector of initial parameters for the
                               optimization
        :param gate_noise: list of Px, Py, Pz probabilities of gate being
                           applied to every gate after each get application
        :param measurement_noise: list of Px', Py', Pz' probabilities of a X, Y
                                  or Z being applied before a measurement.
        :param jacobian: (optional) method of generating jacobian for parameters
                         (Default=None).
        :param qc: (optional) QuantumComputer object.
        :param disp: (optional, bool) display level. If True then each iteration
                     expectation and parameters are printed at each
                     optimization iteration.
        :param samples: (int) Number of samples for calculating the expectation
                        value of the operators.  If `None` then faster method
                        ,dotting the wave function with the operator, is used.
                        Default=None.
        :param return_all: (optional, bool) request to return all intermediate
                           parameters determined during the optimization.
        :return: (vqe.OptResult()) object :func:`OptResult <vqe.OptResult>`.
                 The following fields are initialized in OptResult:
                 -x: set of w.f. ansatz parameters
                 -fun: scalar value of the objective function
                 -iteration_params: a list of all intermediate parameter vectors. Only
                                    returned if 'return_all=True' is set as a vqe_run()
                                    option.
                 -expectation_vals: a list of all intermediate expectation values. Only
                                    returned if 'return_all=True' is set as a
                                    vqe_run() option.
        """
        self._disp_fun = disp if disp is not None else lambda x: None
        iteration_params = []
        expectation_vals = []
        self._current_expectation = None
        if samples is None:
            print("""WARNING: Fast method for expectation will be used. Noise
                     models will be ineffective""")

        if qc is None:
            qubits = hamiltonian.get_qubits()
            qc = QuantumComputer(name=f"{len(qubits)}q-noisy-qvm",
                                 qam=QVM(gate_noise=gate_noise,
                                         measurement_noise=measurement_noise))
        else:
            self.qc = qc

        def objective_function(params):
            """
            closure representing the functional
            :param params: (ndarray) vector of parameters for generating the
                           the function of the functional.
            :return: (float) expectation value
            """
            pyquil_prog = variational_state_evolve(params)
            mean_value = self.expectation(pyquil_prog, hamiltonian, samples, qc)
            self._current_expectation = mean_value  # store for printing
            return mean_value

        def print_current_iter(iter_vars):
            self._disp_fun("\tParameters: {} ".format(iter_vars))
            if jacobian is not None:
                grad = jacobian(iter_vars)
                self._disp_fun("\tGrad-L1-Norm: {}".format(np.max(np.abs(grad))))
                self._disp_fun("\tGrad-L2-Norm: {} ".format(np.linalg.norm(grad)))

            self._disp_fun("\tE => {}".format(self._current_expectation))
            if return_all:
                iteration_params.append(iter_vars)
                expectation_vals.append(self._current_expectation)

        # using self.minimizer
        arguments = funcsigs.signature(self.minimizer).parameters.keys()

        if disp is not None and 'callback' in arguments:
            self.minimizer_kwargs['callback'] = print_current_iter

        args = [objective_function, initial_params]
        args.extend(self.minimizer_args)
        if 'jac' in arguments:
            self.minimizer_kwargs['jac'] = jacobian

        result = self.minimizer(*args, **self.minimizer_kwargs)

        if hasattr(result, 'status'):
            if result.status != 0:
                self._disp_fun("Classical optimization exited with an error index: %i"
                               % result.status)

        results = OptResults()
        if hasattr(result, 'x'):
            results.x = result.x
            results.fun = result.fun
        else:
            results.x = result

        if return_all:
            results.iteration_params = iteration_params
            results.expectation_vals = expectation_vals
        return results

    @staticmethod
    def expectation(pyquil_prog: Program,
                    pauli_sum: Union[PauliSum, PauliTerm, np.ndarray],
                    samples: int,
                    qc: QuantumComputer) -> float:
        """
        Compute the expectation value of pauli_sum over the distribution generated from pyquil_prog.
        :param pyquil_prog: The state preparation Program to calculate the expectation value of.
        :param pauli_sum: PauliSum representing the operator of which to calculate the expectation
            value or a numpy matrix representing the Hamiltonian tensored up to the appropriate
            size.
        :param samples: The number of samples used to calculate the expectation value. If samples
            is None then the expectation value is calculated by calculating <psi|O|psi>. Error
            models will not work if samples is None.
        :param qc: The QuantumComputer object.
        :return: A float representing the expectation value of pauli_sum given the distribution
            generated from quil_prog.
        """
        if isinstance(pauli_sum, np.ndarray):
            # debug mode by passing an array
            wf = WavefunctionSimulator().wavefunction(pyquil_prog)
            wf = np.reshape(wf.amplitudes, (-1, 1))
            average_exp = np.conj(wf).T.dot(pauli_sum.dot(wf)).real
            return average_exp
        else:
            if not isinstance(pauli_sum, (PauliTerm, PauliSum)):
                raise TypeError("pauli_sum variable must be a PauliTerm or PauliSum object")

            if isinstance(pauli_sum, PauliTerm):
                pauli_sum = PauliSum([pauli_sum])

            if samples is None:
                operator_progs = []
                operator_coeffs = []
                for p_term in pauli_sum.terms:
                    op_prog = Program()
                    for qindex, op in p_term:
                        op_prog.inst(STANDARD_GATES[op](qindex))
                    operator_progs.append(op_prog)
                    operator_coeffs.append(p_term.coefficient)

                result_overlaps = WavefunctionSimulator().expectation(pyquil_prog, pauli_sum.terms)
                result_overlaps = list(result_overlaps)
                assert len(result_overlaps) == len(operator_progs),\
                    """Somehow we didn't get the correct number of results back from the QVM"""
                expectation = sum(list(map(lambda x: x[0] * x[1],
                                           zip(result_overlaps, operator_coeffs))))
                return expectation.real
            else:
                if not isinstance(samples, int):
                    raise TypeError("samples variable must be an integer")
                if samples <= 0:
                    raise ValueError("samples variable must be a positive integer")

                # normal execution via fake sampling
                # stores the sum of contributions to the energy from each operator term
                expectation = 0.0
                for j, term in enumerate(pauli_sum.terms):
                    meas_basis_change = Program()
                    qubits_to_measure = []
                    if term.id() == "":
                        meas_outcome = 1.0
                    else:
                        for index, gate in term:
                            qubits_to_measure.append(index)
                            if gate == 'X':
                                meas_basis_change.inst(RY(-np.pi / 2, index))
                            elif gate == 'Y':
                                meas_basis_change.inst(RX(np.pi / 2, index))

                            meas_outcome = \
                                expectation_from_sampling(pyquil_prog + meas_basis_change,
                                                          qubits_to_measure,
                                                          qc,
                                                          samples)

                    expectation += term.coefficient * meas_outcome

                return expectation.real


def parity_even_p(state, marked_qubits):
    """
    Calculates the parity of elements at indexes in marked_qubits
    Parity is relative to the binary representation of the integer state.
    :param state: The wavefunction index that corresponds to this state.
    :param marked_qubits: The indexes to be considered in the parity sum.
    :returns: A boolean corresponding to the parity.
    """
    assert isinstance(state, int), \
        f"{state} is not an integer. Must call parity_even_p with an integer state."
    mask = 0
    for q in marked_qubits:
        mask |= 1 << q
    return bin(mask & state).count("1") % 2 == 0


def expectation_from_sampling(pyquil_program: Program,
                              marked_qubits: List[int],
                              qc: QuantumComputer,
                              samples: int) -> float:
    """
    Calculation of Z_{i} at marked_qubits
    Given a wavefunctions, this calculates the expectation value of the Zi
    operator where i ranges over all the qubits given in marked_qubits.
    :param pyquil_program: pyQuil program generating some state
    :param marked_qubits: The qubits within the support of the Z pauli
                          operator whose expectation value is being calculated
    :param qc: A QuantumComputer object.
    :param samples: Number of bitstrings collected to calculate expectation
                    from sampling.
    :returns: The expectation value as a float.
    """
    program = Program()
    ro = program.declare('ro', 'BIT', max(marked_qubits) + 1)
    program += pyquil_program
    program += [MEASURE(qubit, r) for qubit, r in zip(list(range(max(marked_qubits) + 1)), ro)]
    program.wrap_in_numshots_loop(samples)
    executable = qc.compile(program)
    bitstring_samples = qc.run(executable)
    bitstring_tuples = list(map(tuple, bitstring_samples))

    freq = Counter(bitstring_tuples)

    # perform weighted average
    expectation = 0
    for bitstring, count in freq.items():
        bitstring_int = int("".join([str(x) for x in bitstring[::-1]]), 2)
        if parity_even_p(bitstring_int, marked_qubits):
            expectation += float(count) / samples
        else:
            expectation -= float(count) / samples
    return expectation

### QOAO Helper Class from grove for the QRBM

In [28]:
class QAOA(object):
    def __init__(self, qc: QuantumComputer, qubits: List[int],
                 steps: int = 1,
                 init_betas: List[float] = None,
                 init_gammas: List[float] = None,
                 cost_ham: List[PauliSum] = None,
                 ref_ham: List[PauliSum]= None,
                 driver_ref: Program = None,
                 minimizer: ufunc = None,
                 minimizer_args: List[Any] = None,
                 minimizer_kwargs: Dict[str, Any] = None,
                 rand_seed: int = None,
                 vqe_options = None,
                 store_basis: bool = False):
        """
        QAOA object constructor.
        Contains all information for running the QAOA algorithm to find the
        ground state of the list of cost clauses.
        N.B. This only works if all the terms in the cost Hamiltonian commute with each other.
        :param qc: The QuantumComputer connection to use for the algorithm.
        :param qubits: The list of qubits to use for the algorithm.
        :param steps: The number of mixing and cost function steps to use.
                      Default=1.
        :param init_betas: Initial values for the beta parameters on the
                           mixing terms. Default=None.
        :param init_gammas: Initial values for the gamma parameters on the
                            cost function. Default=None.
        :param cost_ham: list of clauses in the cost function. Must be PauliSum objects.
        :param ref_ham: list of clauses in the mixer function. Must be PauliSum objects.
        :param driver_ref: The object to define state prep
                           for the starting state of the QAOA algorithm.
                           Defaults to tensor product of \|+> states.
        :param minimizer: (Optional) Minimization function to pass to the
                          Variational-Quantum-Eigensolver method.
        :param minimizer_args: (Optional) (list) of additional arguments to pass to the
                               minimizer. Default=[].
        :param minimizer_kwargs: (Optional) (dict) of optional arguments to pass to
                                 the minimizer.  Default={}.
        :param rand_seed: integer random seed for initial betas and gammas
                          guess.
        :param vqe_options: (optional) arguments for VQE run.
        :param store_basis: (optional) boolean flag for storing basis states.
                            Default=False.
        """

        # Seed the random number generator, if a seed is provided.
        if rand_seed is not None:
            np.random.seed(rand_seed)

        # Set attributes values, considering their defaults
        self.qc = qc
        self.steps = steps
        self.qubits = qubits
        self.nstates = 2 ** len(qubits)

        self.cost_ham = cost_ham or []
        self.ref_ham = ref_ham or []

        self.minimizer = minimizer or optimize.minimize
        self.minimizer_args = minimizer_args or []
        self.minimizer_kwargs = minimizer_kwargs or {
            'method': 'Nelder-Mead',
            'options': {
                'disp': True,
                'ftol': 1.0e-2,
                'xtol': 1.0e-2
            }
        }

        self.betas = init_betas or np.random.uniform(0, np.pi, self.steps)[::-1]
        self.gammas = init_gammas or np.random.uniform(0, 2*np.pi, self.steps)
        self.vqe_options = vqe_options or {}

        self.ref_state_prep = (
            driver_ref or
            Program([H(i) for i in self.qubits])
        )

        if store_basis:
            self.states = [
                np.binary_repr(i, width=len(self.qubits))
                for i in range(self.nstates)
            ]

        # Check argument types
        if not isinstance(self.cost_ham, (list, tuple)):
            raise TypeError("cost_ham must be a list of PauliSum objects.")
        if not all([isinstance(x, PauliSum) for x in self.cost_ham]):
            raise TypeError("cost_ham must be a list of PauliSum objects")

        if not isinstance(self.ref_ham, (list, tuple)):
            raise TypeError("ref_ham must be a list of PauliSum objects")
        if not all([isinstance(x, PauliSum) for x in self.ref_ham]):
            raise TypeError("ref_ham must be a list of PauliSum objects")

        if not isinstance(self.ref_state_prep, Program):
            raise TypeError("Please provide a pyQuil Program object "
                            "to generate initial state.")

    def get_parameterized_program(self):
        """
        Return a function that accepts parameters and returns a new Quil program.
        :returns: a function
        """
        cost_para_programs = []
        driver_para_programs = []

        for idx in range(self.steps):
            cost_list = []
            driver_list = []
            for cost_pauli_sum in self.cost_ham:
                for term in cost_pauli_sum.terms:
                    cost_list.append(exponential_map(term))

            for driver_pauli_sum in self.ref_ham:
                for term in driver_pauli_sum.terms:
                    driver_list.append(exponential_map(term))

            cost_para_programs.append(cost_list)
            driver_para_programs.append(driver_list)

        def psi_ref(params):
            """
            Construct a Quil program for the vector (beta, gamma).
            :param params: array of 2 . p angles, betas first, then gammas
            :return: a pyquil program object
            """
            if len(params) != 2*self.steps:
                raise ValueError("params doesn't match the number of parameters set by `steps`")
            betas = params[:self.steps]
            gammas = params[self.steps:]

            prog = Program()
            prog += self.ref_state_prep
            for idx in range(self.steps):
                for fprog in cost_para_programs[idx]:
                    prog += fprog(gammas[idx])

                for fprog in driver_para_programs[idx]:
                    prog += fprog(betas[idx])

            return prog

        return psi_ref

    def get_angles(self) -> Tuple[List[float], List[float]]:
        """
        Finds optimal angles with the quantum variational eigensolver method.
        Stored VQE result
        :returns: A tuple of the beta angles and the gamma angles for the optimal solution.
        """
        stacked_params = np.hstack((self.betas, self.gammas))
        vqe = VQE(self.minimizer, minimizer_args=self.minimizer_args,
                  minimizer_kwargs=self.minimizer_kwargs)
        cost_ham = reduce(lambda x, y: x + y, self.cost_ham)
        # maximizing the cost function!
        param_prog = self.get_parameterized_program()
        result = vqe.vqe_run(param_prog, cost_ham, stacked_params, qc=self.qc,
                             **self.vqe_options)
        self.result = result
        betas = result.x[:self.steps]
        gammas = result.x[self.steps:]
        return betas, gammas

    def probabilities(self, angles: List[float]) -> np.ndarray:
        """
        Computes the probability of each state given a particular set of angles.
        :param angles: A concatenated list of angles [betas]+[gammas]
        :return: The probabilities of each outcome given those angles.
        """
        if isinstance(angles, list):
            angles = np.array(angles)

        assert angles.shape[0] == 2 * self.steps, "angles must be 2 * steps"

        param_prog = self.get_parameterized_program()
        prog = param_prog(angles)
        wf = WavefunctionSimulator().wavefunction(prog)
        wf = wf.amplitudes.reshape((-1, 1))
        probs = np.zeros_like(wf)
        for xx in range(2 ** len(self.qubits)):
            probs[xx] = np.conj(wf[xx]) * wf[xx]
        return probs

    def get_string(self, betas: List[float], gammas: List[float], samples: int = 100):
        """
        Compute the most probable string.
        The method assumes you have passed init_betas and init_gammas with your
        pre-computed angles or you have run the VQE loop to determine the
        angles.  If you have not done this you will be returning the output for
        a random set of angles.
        :param betas: List of beta angles
        :param gammas: List of gamma angles
        :param samples: (Optional) number of samples to get back from the QuantumComputer.
        :returns: tuple representing the bitstring, Counter object from
                  collections holding all output bitstrings and their frequency.
        """
        if samples <= 0 and not isinstance(samples, int):
            raise ValueError("samples variable must be positive integer")
        param_prog = self.get_parameterized_program()
        stacked_params = np.hstack((betas, gammas))

        sampling_prog = Program()
        ro = sampling_prog.declare('ro', 'BIT', len(self.qubits))
        sampling_prog += param_prog(stacked_params)
        sampling_prog += [MEASURE(qubit, r) for qubit, r in zip(self.qubits, ro)]
        sampling_prog.wrap_in_numshots_loop(samples)
        executable = self.qc.compile(sampling_prog)
        bitstring_samples = self.qc.run(executable)
        bitstring_tuples = list(map(tuple, bitstring_samples))
        freq = Counter(bitstring_tuples)
        most_frequent_bit_string = max(freq, key=lambda x: freq[x])
        return most_frequent_bit_string, freq

In [29]:
def make_qvm(qvm=None):
    if qvm:
        return qvm
    else:
        return api.QVMConnection()

## Quantum Classical Hybrid Restricted Boltzmann Machine Implementation

In [33]:
class QBM:
    """
    Quantum Classical Hybrid RBM implementation.
    """
    def __init__(self, qvm=None, num_visible=2, num_hidden=1, steps=3, temp=1.0, quant_meas_num=None, bias=False, reduced=False):
        # Initializing the Params
        self.visible_units = num_visible #Number of visible units
        self.hidden_units = num_hidden #Number of hidden units
        self.total_units = self.visible_units + self.hidden_units
        self.qvm = make_qvm(qvm) #Rigetti QVM Connection Simulator
        self.quant_meas_num = quant_meas_num # Number of measuremants to use for Quantum expectation estimation
        self.qaoa_steps = steps #Number of steps for QAOA
        self.beta_temp = temp #Temperature of the system
        self.state_prep_angle = np.arctan(np.exp(-1/self.beta_temp)) * 2.0
        self.vqe_inst = VQE(minimizer=minimize, minimizer_kwargs={'method': 'nelder-mead'}) #VQE to minimize the value
        self.param_wb = 0.1 * np.sqrt(6. / self.total_units)
        self.WEIGHTS = np.asarray(np.random.uniform(low=-self.param_wb, high=self.param_wb, size=(num_visible, num_hidden)))

        # Using Bias or not.
        if bias:
            self.BIAS = np.asarray(np.random.uniform(low=-self.param_wb, high=self.param_wb,size=(self.hidden_units)))
        else:
            self.BIAS = None

        # Using Reduced or Full Botlzman machines.
        if reduced:
            self.reduced = True
        else:
            self.reduced = False
        
    def make_clamped_QAOA(self, data_point, iter):
        """
        Building Quantum Approximate Optimization Algorithm circuit to get the Restricted Boltzmann Machine expectation using a clamped approach
        """
        visible_indices = [i for i in range(self.visible_units)]
        hidden_indices = [i + self.visible_units for i in range(self.hidden_units)]
        total_indices = [i for i in range(self.total_units)]
        # Partial Mixer and Partial Cost Hamiltonian
        partial_mixer_operator = []
        for i in hidden_indices:
            partial_mixer_operator.append(PauliSum([PauliTerm("X", i, 1.0)]))
        partial_cost_operator = []
        for i in visible_indices:
            for j in hidden_indices:
                partial_cost_operator.append(PauliSum([PauliTerm(
                    "Z", i, -1.0 * self.WEIGHTS[i][j - self.visible_units]) * PauliTerm("Z", j, 1.0)]))
        if self.BIAS is not None:
            for i in hidden_indices:
                partial_cost_operator.append(
                    PauliSum([PauliTerm("Z", i, -1.0 * self.BIAS[i - self.visible_units])]))
        state_prep = pq.Program()
        for i, j in enumerate(data_point):
            if j == 1:
                state_prep += X(i)

        for i in hidden_indices:
            tmp = pq.Program()
            tmp.inst(RX(self.state_prep_angle, i + self.total_units),
                     CNOT(i + self.total_units, i))
            state_prep += tmp
        partial_QAOA = QAOA(qc=self.qvm,
                   qubits=total_indices,
                   steps=self.qaoa_steps,
                   ref_ham=partial_mixer_operator,
                   cost_ham=partial_cost_operator,
                   driver_ref=state_prep,
                   store_basis=True,
                   minimizer=fmin_bfgs,
                   minimizer_kwargs={'maxiter': 100//iter},
                   vqe_options={'samples': self.quant_meas_num},
                    rand_seed=1234)
                    
        nus, gammas = partial_QAOA.get_angles()
        program = partial_QAOA.get_parameterized_program()
        return nus, gammas, program, 1

    def make_unclamped_QAOA(self):
        """
        Building Quantum Approximate Optimization Algorithm circuit to get the Restricted Boltzmann Machine expectation using an unclamped approach.
        """
        visible_indices = [i for i in range(self.visible_units)]
        hidden_indices = [i + self.visible_units for i in range(self.hidden_units)]
        total_indices = [i for i in range(self.total_units)]
        # Full Mixer and Cost Hamiltonian Operator
        full_mixer_operator = []
        for i in total_indices:
            full_mixer_operator.append(PauliSum([PauliTerm("X", i, 1.0)]))
        full_cost_operator = []
        for i in visible_indices:
            for j in hidden_indices:
                full_cost_operator.append(PauliSum([PauliTerm(
                    "Z", i, -1.0 * self.WEIGHTS[i][j - self.visible_units]) * PauliTerm("Z", j, 1.0)]))
        if self.BIAS is not None:
            for i in hidden_indices:
                print(i, self.visible_units, i-self.visible_units, self.BIAS[i-self.visible_units])
                full_cost_operator.append(
                    PauliSum([PauliTerm("Z", i, -1.0 * self.BIAS[i-self.visible_units])]))

        # Prepare all the units in a thermal state of the full mixer hamiltonian.         
        state_prep = pq.Program()
        for i in total_indices:
            tmp = pq.Program()
            tmp.inst(RX(self.state_prep_angle, i + self.total_units), CNOT(i + self.total_units, i))
            state_prep += tmp
        # QAOA on full mixer and full cost hamiltonian evolution
        full_QAOA = QAOA(self.qvm,
                   qubits=total_indices,
                   steps=self.qaoa_steps,
                   ref_ham=full_mixer_operator,
                   cost_ham=full_cost_operator,
                   driver_ref=state_prep,
                   store_basis=True,
                   minimizer=fmin_bfgs,
                   minimizer_kwargs={'maxiter': 100},
                   vqe_options={'samples': self.quant_meas_num},
                    rand_seed=1234)

        nus, gammas = full_QAOA.get_angles()
        program = full_QAOA.get_parameterized_program()
        return nus, gammas, program, 0

    def sigmoid(self, x):
        """
        Returns the sigmoid function required for training.
        """
        return 1.0 / (1.0 + np.exp(-x))

    

## Setting the Training Parameters for the Algorithm

In [52]:
DATA = [[1, 1, -1, -1], [1, 1, -1, -1], [-1, -1, 1, 1], [-1, -1, 1, 1]]
n_epochs= 10
quantum_percentage= 0.7
classical_percentage = 0.3
qvm = api.QVMConnection()

Qalg = QBM(qvm, num_visible=4, num_hidden=1, quant_meas_num=None, bias=False, reduced=False)

### Training for the Quantum Restrictive Boltzmann Machine

In [54]:
assert(quantum_percentage+ classical_percentage == 1.0)
DATA = np.asarray(DATA)
assert(len(DATA[0]) <= Qalg.visible_units)

#Training
for epoch in range(n_epochs):
    print("Epoch: ", epoch)
    visible_indices = [i for i in range(Qalg.visible_units)]
    hidden_indices = [i + Qalg.visible_units for i in range(Qalg.hidden_units)]
    total_indices = [i for i in range(Qalg.total_units)]

    new_weights = deepcopy(Qalg.WEIGHTS)
    if Qalg.BIAS is not None:
        new_bias = deepcopy(Qalg.BIAS)
    unc_nus, unc_gammas, unc_para_prog, _ = Qalg.make_unclamped_QAOA()
    unc_mod_samp_prog = unc_para_prog(np.hstack((unc_nus, unc_gammas)))

    print('Found model expectation program')

    unc_neg_phase_quant = np.zeros_like(Qalg.WEIGHTS)

    for i in range(Qalg.visible_units):
        for j in range(Qalg.hidden_units):
            model_expectation = Qalg.vqe_inst.expectation(unc_mod_samp_prog, sZ(visible_indices[i]) * sZ(hidden_indices[j]), Qalg.quant_meas_num, Qalg.qvm)
            unc_neg_phase_quant[i][j] = model_expectation
            
    unc_neg_phase_quant  *= (1. / float(len(DATA)))

    if Qalg.BIAS is not None:
        unc_neg_phase_quant_bias = np.zeros_like(Qalg.BIAS)
        for i in range(Qalg.hidden_units):
            model_expectation = Qalg.vqe_inst.expectation(unc_mod_samp_prog, sZ(hidden_indices[i]), Qalg.quant_meas_num, Qalg.qvm)
            unc_neg_phase_quant_bias[i] = model_expectation
        unc_neg_phase_quant_bias *= (1. / float(len(DATA)))

    pos_hidden_probs = Qalg.sigmoid(np.dot(DATA, Qalg.WEIGHTS))
    pos_hidden_states = pos_hidden_probs > np.random.rand(len(DATA), Qalg.hidden_units)
    pos_phase_classical = np.dot(DATA.T, pos_hidden_probs) * 1./len(DATA)
    c_pos_phase_quant = np.zeros_like(Qalg.WEIGHTS)

    if Qalg.BIAS is not None:
        c_pos_phase_quant_bias = np.zeros_like(Qalg.BIAS)

    if not Qalg.reduced:
        iter_dat= len(DATA)
        for data in DATA:
            c_nus, c_gammas, c_para_prog, _ = Qalg.make_clamped_QAOA(data_point=data, iter= iter_dat)
            c_mod_samp_prog = c_para_prog(np.hstack((c_nus, c_gammas)))

            print('Found Model Expectation')

            ct_pos_phase_quant = np.zeros_like(Qalg.WEIGHTS)

            for i in range(Qalg.visible_units):
                for j in range(Qalg.hidden_units):
                    model_expectation = Qalg.vqe_inst.expectation(c_mod_samp_prog,sZ(visible_indices[i]) * sZ(hidden_indices[j]), Qalg.quant_meas_num, Qalg.qvm)
                    ct_pos_phase_quant[i][j] = model_expectation
            c_pos_phase_quant += ct_pos_phase_quant

            if Qalg.BIAS is not None:
                ct_pos_phase_quant_bias = np.zeros_like(Qalg.BIAS)
                for i in range(Qalg.hidden_units):
                    model_expectation = Qalg.vqe_inst.expectation(c_mod_samp_prog, sZ(hidden_indices[j]), Qalg.quant_meas_num, Qalg.qvm)
                    ct_pos_phase_quant_bias[i] = model_expectation
                c_pos_phase_quant_bias += ct_pos_phase_quant_bias
                
        c_pos_phase_quant *= (1. / float(len(DATA)))
        if Qalg.BIAS is not None:
            c_pos_phase_quant_bias *= (1. / float(len(DATA)))

    learning_rate=0.1
    neg_visible_activations = np.dot(pos_hidden_states, Qalg.WEIGHTS.T)
    neg_visible_probs = Qalg.sigmoid(neg_visible_activations)
    neg_hidden_activations = np.dot(neg_visible_probs, Qalg.WEIGHTS)
    neg_hidden_probs = Qalg.sigmoid(neg_hidden_activations)
    neg_phase_classical = np.dot(neg_visible_probs.T, neg_hidden_probs) * 1./len(DATA)
    new_weights += learning_rate * (classical_percentage * (pos_phase_classical - neg_phase_classical) + quantum_percentage * (c_pos_phase_quant - unc_neg_phase_quant))

    print(Qalg.BIAS)

    Qalg.WEIGHTS = deepcopy(new_weights)

    if Qalg.BIAS is not None:
        Qalg.BIAS = deepcopy(new_bias)
        print(Qalg.BIAS)

    print('Training Done')


print("The Weights Determined by the algorithm are")
print(Qalg.WEIGHTS)

if Qalg.BIAS is not None:
    print(Qalg.BIAS)

# To save the discovered data
with open("RBM_info.txt", "w") as f: 
    np.savetxt(f,Qalg.WEIGHTS)
    if Qalg.BIAS is not None:
        np.savetxt(f,Qalg.BIAS)


Epoch:  0
                     models will be ineffective
Optimization terminated successfully.
         Current function value: -0.030299
         Iterations: 41
         Function evaluations: 308
         Gradient evaluations: 44
Found model expectation program
                     models will be ineffective
Optimization terminated successfully.
         Current function value: -0.006074
         Iterations: 10
         Function evaluations: 91
         Gradient evaluations: 13
Found Model Expectation
                     models will be ineffective
Optimization terminated successfully.
         Current function value: -0.006074
         Iterations: 10
         Function evaluations: 91
         Gradient evaluations: 13
Found Model Expectation
                     models will be ineffective
Optimization terminated successfully.
         Current function value: -0.006074
         Iterations: 7
         Function evaluations: 98
         Gradient evaluations: 14
Found Model Expectation
  

In [55]:
print("Actual results are: \n \n")

def transform(DATA):
    return Qalg.sigmoid(np.dot(DATA, Qalg.WEIGHTS))

print(transform(DATA))

Actual results are: 
 

[[0.1359068]
 [0.1359068]
 [0.8640932]
 [0.8640932]]
